In [21]:
import numpy as np
import pandas as pd
from math import log

In [22]:
df = pd.read_csv('./example_data.csv')
df

,humility,outlook,play,temp,windy
0,high,sunny,no,hot,False
1,high,sunny,no,hot,True
2,high,overcast,yes,hot,False
3,high,rainy,yes,mild,False
4,normal,rainy,yes,cool,False
5,normal,rainy,no,cool,True
6,normal,overcast,yes,cool,True
7,high,sunny,no,mild,False
8,normal,sunny,yes,cool,False
9,normal,rainy,yes,mild,False


In [23]:
def entropy(ele):
    '''
    function: Calculating entropy value.
    input: A list contain categorical value.
    output: Entropy value.
    entropy = - sum(p * log(p)), p is a prob value.
    '''
    # Calculating the probability distribution of list value
    probs = [ele.count(i)/len(ele) for i in set(ele)]
    # Calculating entropy value
    entropy = -sum([prob*log(prob, 2) for prob in probs])
    return entropy

In [24]:
entropy(df['play'].tolist())

0.9402859586706309

In [25]:
def split_dataframe(data, col):
    '''
    function: split pandas dataframe to sub-df based on data and column.
    input: dataframe, column name.
    output: a dict of splited dataframe.
    '''
    # unique value of column
    unique_values = data[col].unique()
    # empty dict of dataframe
    result_dict = {elem : pd.DataFrame for elem in unique_values} #特征列col上的所有特征值，做诶

    print (result_dict)
    # split dataframe based on column value
    for key in result_dict.keys():
        result_dict[key] = data[:][data[col] == key]   ## ?? 这个地方没看懂
    return result_dict

In [26]:
split_example = split_dataframe(df, 'temp')
split_example

{'hot': <class 'pandas.core.frame.DataFrame'>, 'mild': <class 'pandas.core.frame.DataFrame'>, 'cool': <class 'pandas.core.frame.DataFrame'>}


{'hot':    humility   outlook play temp  windy
 0      high     sunny   no  hot  False
 1      high     sunny   no  hot   True
 2      high  overcast  yes  hot  False
 12   normal  overcast  yes  hot  False,
 'mild':    humility   outlook play  temp  windy
 3      high     rainy  yes  mild  False
 7      high     sunny   no  mild  False
 9    normal     rainy  yes  mild  False
 10   normal     sunny  yes  mild   True
 11     high  overcast  yes  mild   True
 13     high     rainy   no  mild   True,
 'cool':   humility   outlook play  temp  windy
 4   normal     rainy  yes  cool  False
 5   normal     rainy   no  cool   True
 6   normal  overcast  yes  cool   True
 8   normal     sunny  yes  cool  False}

In [27]:
for item, value in split_example.items():
    print(item, value)

hot    humility   outlook play temp  windy
0      high     sunny   no  hot  False
1      high     sunny   no  hot   True
2      high  overcast  yes  hot  False
12   normal  overcast  yes  hot  False
mild    humility   outlook play  temp  windy
3      high     rainy  yes  mild  False
7      high     sunny   no  mild  False
9    normal     rainy  yes  mild  False
10   normal     sunny  yes  mild   True
11     high  overcast  yes  mild   True
13     high     rainy   no  mild   True
cool   humility   outlook play  temp  windy
4   normal     rainy  yes  cool  False
5   normal     rainy   no  cool   True
6   normal  overcast  yes  cool   True
8   normal     sunny  yes  cool  False


In [28]:
def choose_best_col(df, label):
    '''
    funtion: choose the best column based on infomation gain.
    input: datafram, label
    output: max infomation gain, best column, 
            splited dataframe dict based on best column.
    '''
    # Calculating label's entropy
    entropy_D = entropy(df[label].tolist())
    # columns list except label
    cols = [col for col in df.columns if col not in [label]]
    # initialize the max infomation gain, best column and best splited dict
    max_value, best_col = -999, None
    max_splited = None
    # split data based on different column
    for col in cols:
        splited_set = split_dataframe(df, col)
        entropy_DA = 0
        for subset_col, subset in splited_set.items():
            # calculating splited dataframe label's entropy
            entropy_Di = entropy(subset[label].tolist())
            # calculating entropy of current feature
            entropy_DA += len(subset)/len(df) * entropy_Di
        # calculating infomation gain of current feature
        info_gain = entropy_D - entropy_DA
        
        if info_gain > max_value:
            max_value, best_col = info_gain, col
            max_splited = splited_set
    return max_value, best_col, max_splited
    
choose_best_col(df, 'play')

{'high': <class 'pandas.core.frame.DataFrame'>, 'normal': <class 'pandas.core.frame.DataFrame'>}
{'sunny': <class 'pandas.core.frame.DataFrame'>, 'overcast': <class 'pandas.core.frame.DataFrame'>, 'rainy': <class 'pandas.core.frame.DataFrame'>}
{'hot': <class 'pandas.core.frame.DataFrame'>, 'mild': <class 'pandas.core.frame.DataFrame'>, 'cool': <class 'pandas.core.frame.DataFrame'>}
{False: <class 'pandas.core.frame.DataFrame'>, True: <class 'pandas.core.frame.DataFrame'>}


(0.2467498197744391,
 'outlook',
 {'sunny':    humility outlook play  temp  windy
  0      high   sunny   no   hot  False
  1      high   sunny   no   hot   True
  7      high   sunny   no  mild  False
  8    normal   sunny  yes  cool  False
  10   normal   sunny  yes  mild   True,
  'overcast':    humility   outlook play  temp  windy
  2      high  overcast  yes   hot  False
  6    normal  overcast  yes  cool   True
  11     high  overcast  yes  mild   True
  12   normal  overcast  yes   hot  False,
  'rainy':    humility outlook play  temp  windy
  3      high   rainy  yes  mild  False
  4    normal   rainy  yes  cool  False
  5    normal   rainy   no  cool   True
  9    normal   rainy  yes  mild  False
  13     high   rainy   no  mild   True})

In [29]:
class ID3Tree:
    # define a Node class
    class Node:
        def __init__(self, name):
            self.name = name
            self.connections = {}

        def connect(self, label, node):
            self.connections[label] = node
            
    def __init__(self, data, label):
        self.columns = data.columns
        self.data = data
        self.label = label
        self.root = self.Node("Root")
    
    # print tree method
    def print_tree(self, node, tabs):
        print(tabs + node.name)
        for connection, child_node in node.connections.items():
            print(tabs + "\t" + "(" + connection + ")")
            self.print_tree(child_node, tabs + "\t\t")
    
    def construct_tree(self):
        self.construct(self.root, "", self.data, self.columns)
    
    # construct tree
    def construct(self, parent_node, parent_connection_label, input_data, columns):
        max_value, best_col, max_splited = choose_best_col(input_data[columns], self.label)
        
        if not best_col:
            node = self.Node(input_data[self.label].iloc[0])
            parent_node.connect(parent_connection_label, node)
            return

        node = self.Node(best_col)
        parent_node.connect(parent_connection_label, node)
        
        new_columns = [col for col in columns if col != best_col]
        
        # Recursively constructing decision trees
        for splited_value, splited_data in max_splited.items():
            self.construct(node, splited_value, splited_data, new_columns)

In [30]:
tree1 = ID3Tree(df, 'play')
tree1.construct_tree()
tree1.print_tree(tree1.root, "")

{'high': <class 'pandas.core.frame.DataFrame'>, 'normal': <class 'pandas.core.frame.DataFrame'>}
{'sunny': <class 'pandas.core.frame.DataFrame'>, 'overcast': <class 'pandas.core.frame.DataFrame'>, 'rainy': <class 'pandas.core.frame.DataFrame'>}
{'hot': <class 'pandas.core.frame.DataFrame'>, 'mild': <class 'pandas.core.frame.DataFrame'>, 'cool': <class 'pandas.core.frame.DataFrame'>}
{False: <class 'pandas.core.frame.DataFrame'>, True: <class 'pandas.core.frame.DataFrame'>}
{'high': <class 'pandas.core.frame.DataFrame'>, 'normal': <class 'pandas.core.frame.DataFrame'>}
{'hot': <class 'pandas.core.frame.DataFrame'>, 'mild': <class 'pandas.core.frame.DataFrame'>, 'cool': <class 'pandas.core.frame.DataFrame'>}
{False: <class 'pandas.core.frame.DataFrame'>, True: <class 'pandas.core.frame.DataFrame'>}
{'hot': <class 'pandas.core.frame.DataFrame'>, 'mild': <class 'pandas.core.frame.DataFrame'>}
{False: <class 'pandas.core.frame.DataFrame'>, True: <class 'pandas.core.frame.DataFrame'>}
{False

TypeError: can only concatenate str (not "numpy.bool_") to str